# Пример использования сценария

Лаботоратория по искусственному интеллекту, Сбербанк. 

О чем: вызов сценариев с разными моделями.
В качестве датасета используется датасет MovieLens100K.

### Импорты, создание спарк-сессии <a name='intro'></a>

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from sponge_bob_magic.session_handler import State

spark = State().session
spark

## Загрузка данных <a name="data-loader"></a>

In [3]:
# используем пакет https://pypi.org/project/rs-datasets/
!pip install rs_datasets

In [4]:
from rs_datasets import MovieLens

data = MovieLens("25m")
data.info()

ratings


,user_id,item_id,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828



items


,item_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance



tags


,user_id,item_id,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598



links


,item_id,imdb_id,tmdb_id
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0


In [5]:
# загрузим данные в Spark

from sponge_bob_magic.data_preparator import DataPreparator

log = DataPreparator().transform(
    data=data.ratings,
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "timestamp": "timestamp",
        "relevance": "rating"
    }
).cache()

## Сценарии с разными моделями <a name="scenario"></a>

### Получение сценария через фабрику <a name="get-scenario"></a>

In [6]:
from sponge_bob_magic.models import ALSWrap, SLIM

als = ALSWrap()
slim = SLIM()

In [7]:
from sponge_bob_magic.splitters import UserSplitter

user_random_splitter = UserSplitter(
    item_test_size=1,
    user_test_size=10000,
    drop_cold_items=True,
    drop_cold_users=True,
    shuffle=True,
    seed=1234
)

### Обучение сценария <a name="fit-scenario"></a>

In [8]:
from sponge_bob_magic.scenarios import MainScenario
from sponge_bob_magic.metrics import NDCG, HitRate

scenario = MainScenario(
    splitter=user_random_splitter,
    recommender=als,
    criterion=HitRate,
    metrics={
        NDCG: [10, 5, 1],
        HitRate: [10, 5, 1],
    }
)

In [9]:
scenario.research(
    {"rank": [2 ** i for i in range(10)]},
    log,
    k=10,
    n_trials=10
)
als_results = scenario.experiment.pandas_df

27-Apr-20 17:28:07, sponge_bob_magic, DEBUG: Деление лога на обучающую и тестовую выборку
27-Apr-20 17:28:35, sponge_bob_magic, DEBUG: Длина трейна и теста: 24990095 9998
27-Apr-20 17:28:36, sponge_bob_magic, DEBUG: Количество пользователей в трейне и тесте: 162541, 9998
27-Apr-20 17:28:40, sponge_bob_magic, DEBUG: Количество объектов в трейне и тесте: 59045, 2849
27-Apr-20 17:28:40, sponge_bob_magic, DEBUG: Инициализация метрик
27-Apr-20 17:28:40, sponge_bob_magic, DEBUG: Обучение и предсказание дополнительной модели
27-Apr-20 17:28:40, sponge_bob_magic, DEBUG: Предварительная стадия обучения (pre-fit)
27-Apr-20 17:28:46, sponge_bob_magic, DEBUG: Основная стадия обучения (fit)
/home/shminke-ba/projects/sponge-bob-magic/venv/lib/python3.6/site-packages/pyarrow/util.py:39: FutureWarning:

pyarrow.open_stream is deprecated as of 0.17.0, please use pyarrow.ipc.open_stream instead

27-Apr-20 17:28:55, sponge_bob_magic, DEBUG: Пре-фит модели
27-Apr-20 17:28:59, sponge_bob_magic, DEBUG: Опти

27-Apr-20 21:38:13, sponge_bob_magic, DEBUG: Обучение модели
27-Apr-20 21:46:29, sponge_bob_magic, DEBUG: -- Предикт модели в оптимизации
27-Apr-20 21:46:51, sponge_bob_magic, DEBUG: -- Длина рекомендаций: 99980
27-Apr-20 21:46:51, sponge_bob_magic, DEBUG: -- Добавляем fallback рекомендации
27-Apr-20 21:46:54, sponge_bob_magic, DEBUG: -- Длина рекомендаций после добавления fallback-рекомендаций: 99980
27-Apr-20 21:46:54, sponge_bob_magic, DEBUG: -- Подсчет метрики в оптимизации
27-Apr-20 21:47:38, sponge_bob_magic, DEBUG: HitRate=0.28
[I 2020-04-27 21:47:38,180] Finished trial#8 with value: 0.2816563312662532 with parameters: {'rank': 128}. Best is trial#6 with value: 0.3049609921984397.
27-Apr-20 21:47:38, sponge_bob_magic, DEBUG: -- Второй фит модели в оптимизации
27-Apr-20 21:47:38, sponge_bob_magic, DEBUG: Индексирование данных
27-Apr-20 21:47:38, sponge_bob_magic, DEBUG: Обучение модели
27-Apr-20 21:48:06, sponge_bob_magic, DEBUG: -- Предикт модели в оптимизации
27-Apr-20 21:48:21

In [10]:
scenario.recommender = slim
scenario.research(
    {
        "beta": [2 ** i for i in range(-8, -4)],
        "lambda_": [2 ** i for i in range(-8, -4)],
    },
    log,
    k=10,
    n_trials=10
)
slim_results = scenario.experiment.pandas_df

27-Apr-20 21:49:09, sponge_bob_magic, DEBUG: Деление лога на обучающую и тестовую выборку
27-Apr-20 21:49:10, sponge_bob_magic, DEBUG: Длина трейна и теста: 24990095 9998
27-Apr-20 21:49:11, sponge_bob_magic, DEBUG: Количество пользователей в трейне и тесте: 162541, 9998
27-Apr-20 21:49:12, sponge_bob_magic, DEBUG: Количество объектов в трейне и тесте: 59045, 2849
27-Apr-20 21:49:12, sponge_bob_magic, DEBUG: Инициализация метрик
27-Apr-20 21:49:12, sponge_bob_magic, DEBUG: Обучение и предсказание дополнительной модели
27-Apr-20 21:49:12, sponge_bob_magic, DEBUG: Предварительная стадия обучения (pre-fit)
27-Apr-20 21:49:15, sponge_bob_magic, DEBUG: Основная стадия обучения (fit)
27-Apr-20 21:49:21, sponge_bob_magic, DEBUG: Пре-фит модели
27-Apr-20 21:49:24, sponge_bob_magic, DEBUG: Оптимизация параметров
27-Apr-20 21:49:24, sponge_bob_magic, DEBUG: Количество попыток: 10
27-Apr-20 21:49:31, sponge_bob_magic, DEBUG: -- Второй фит модели в оптимизации
27-Apr-20 21:49:31, sponge_bob_magic,

28-Apr-20 00:05:21, sponge_bob_magic, DEBUG: Построение модели SLIM
28-Apr-20 00:06:22, sponge_bob_magic, DEBUG: -- Предикт модели в оптимизации
28-Apr-20 00:21:25, sponge_bob_magic, DEBUG: -- Длина рекомендаций: 99980
28-Apr-20 00:21:25, sponge_bob_magic, DEBUG: -- Добавляем fallback рекомендации
28-Apr-20 00:21:27, sponge_bob_magic, DEBUG: -- Длина рекомендаций после добавления fallback-рекомендаций: 99980
28-Apr-20 00:21:27, sponge_bob_magic, DEBUG: -- Подсчет метрики в оптимизации
28-Apr-20 00:22:01, sponge_bob_magic, DEBUG: HitRate=0.23
[I 2020-04-28 00:22:01,393] Finished trial#9 with value: 0.23434686937387478 with parameters: {'beta': 0.03125, 'lambda_': 0.0078125}. Best is trial#7 with value: 0.2401480296059212.
28-Apr-20 00:22:01, sponge_bob_magic, DEBUG: Лучшее значение метрики: 0.24
28-Apr-20 00:22:01, sponge_bob_magic, DEBUG: Лучшие параметры: {'beta': 0.0078125, 'lambda_': 0.03125}


In [12]:
import pandas as pd

pd.set_option("display.max_rows", 20)
slim_results.append(als_results).sort_values("HitRate@10")

,HitRate@10,HitRate@1,HitRate@5,NDCG@1,NDCG@5,NDCG@10
ALSWrap(rank=1),0.082316,0.017403,0.051610,0.017403,0.034504,0.044433
ALSWrap(rank=2),0.117323,0.029406,0.076315,0.029406,0.053221,0.066408
ALSWrap(rank=4),0.135527,0.030006,0.088418,0.030006,0.059364,0.074547
ALSWrap(rank=8),0.178236,0.043209,0.122625,0.043209,0.083422,0.101318
ALSWrap(rank=16),0.206041,0.052611,0.143329,0.052611,0.098078,0.118270
"SLIM(lambda_=0.00390625, beta=0.0078125)",0.230746,0.052611,0.156231,0.052611,0.105188,0.129119
ALSWrap(rank=32),0.230946,0.062312,0.164333,0.062312,0.114443,0.135843
"SLIM(lambda_=0.0078125, beta=0.0078125)",0.232446,0.054411,0.158432,0.054411,0.107295,0.131098
"SLIM(lambda_=0.00390625, beta=0.03125)",0.232947,0.053111,0.157832,0.053111,0.106238,0.130390
"SLIM(lambda_=0.0078125, beta=0.015625)",0.233147,0.054311,0.158732,0.054311,0.107339,0.131295


### Получение рекомендаций <a name="predict-scenario"></a>

In [ ]:
recs = scenario.production(best_params, log, k=10)

In [ ]:
recs.show()